In [18]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import math

In [19]:
def scrape_page(url):
    # response = requests.get(url, params={"cursor": cursor})
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    return soup

In [20]:
def get_cursor(soup, next_page_num):
    cursor = soup.find('a', {'class':'chakra-link css-1wxsdwr', 'aria-label': f'page {next_page_num}'})
    if cursor:
        return cursor.get('href')
    else:
        return None

In [21]:
def get_job_links(url, list_elems):
    job_links = []
    for item in list_elems:
        job_key = item.div.get('data-jobkey')
        job_link = f'{url}&job={job_key}'
        job_links.append(job_link)
    return job_links


In [22]:
def scrape_one_page(url, soup):
    ul = soup.find_all('ul', id = 'job-list')
    li = ul[0].find_all('li', {'class':'css-0'})
    job_links = get_job_links(url, li)
    data_list = []
    for job_link in job_links:
        print(job_link)
        job = scrape_page(job_link)
        job_title = job.h2.text.strip()
        company = job.find('span', {'data-testid': 'viewJobCompanyName'}).text if job.find('span', {'data-testid': 'viewJobCompanyName'}) else ""
        location = job.find('span', {'data-testid': 'viewJobCompanyLocation'}).text if job.find('span', {'data-testid': 'viewJobCompanyLocation'}) else ""
        job_type = job.find('span', {'data-testid': 'viewJobBodyJobDetailsJobType'}).text if job.find('span', {'data-testid': 'viewJobBodyJobDetailsJobType'}) else ""
        salary = job.find('span', {'data-testid': 'viewJobBodyJobCompensation'}).text if job.find('span', {'data-testid': 'viewJobBodyJobCompensation'}) else ""
        posted_on = job.find('span', {'data-testid': 'viewJobBodyJobPostingTimestamp'}).text if job.find('span', {'data-testid': 'viewJobBodyJobPostingTimestamp'}) else ""
        job_qualification = job.find('div', {'data-testid': 'viewJobQualificationsContainer'})\
                                .find('ul')
        if job_qualification:
            job_qualification = list(job.find('div', {'data-testid': 'viewJobQualificationsContainer'})\
                                    .find('ul').strings)
            job_qualification = [x.strip() for x in job_qualification]
            job_qualification = "\n".join(job_qualification)
        else:
            job_qualification = ""

        job_description = job.find('div', {'data-testid':'viewJobBodyJobFullDescriptionContent'})
        if job_description:
            job_description = list(job.find('div', {'data-testid':'viewJobBodyJobFullDescriptionContent'}).strings)
            job_description = [x.strip() for x in job_description]
            job_description = "\n".join(job_description)
        else:
            job_description = ""
        data = {
                'job_title':job_title,
                'company':company,
                'location': location, 
                'job_type': job_type, 
                'salary':salary, 
                'posted_on':posted_on, 
                'job_qualification':job_qualification, 
                'job_description': job_description
                }
        data_list.append(data)
        # break
    return data_list
        

In [24]:
job = scrape_page('https://www.simplyhired.com/search?q=Data+Engineer&l=texas&cursor=AAsAAwAhAAAAAAAAAAAAAAACCRy%2B%2BQECAQYQBgb%2FwPwDEb8bt6M7md5zCuPpuHutpd9TqtvNULnns4VOaMsY9BbfxiZtadJD3rA1JRnbckZ3vZMjS6z6fA%3D%3D&job=ThYj28cCvjeaNFzLrRz-kCYzKA3eW110q-c4D8XJIwxIDQqX61IXvg')

job_qualification = job.find('div', {'data-testid': 'viewJobQualificationsContainer'})
if job_qualification:
    job_qualification = list(job.find('div', {'data-testid': 'viewJobQualificationsContainer'})\
                            .find('ul').strings)
    job_qualification = [x.strip() for x in job_qualification]
    job_qualification = "\n".join(job_qualification)
else:
    job_qualification = ""

print("Here", job_qualification)

Here Jira
SFTP
Computer Science
Alteryx
AWS
Bachelor's degree
Terraform
Computer Information Systems
S3
Redshift
DynamoDB
Jenkins
Python
MySQL


In [ ]:
# import pandas as pd
# job = 'Data Engineer'
# location = 'texas'
# cursor = ""
# url = f'https://www.simplyhired.com/search?q={job}&l={location}'
# df = pd.DataFrame(columns=['job_title', 'company', 'location', 'job_type', 'salary', 'posted_on', 'job_qualification', 'job_description'])
# soup = scrape_page(url)
# page_data = scrape_one_page(url, soup)
# df = df.append(page_data)


# while True:
#     next_page = get_cursor(soup)
#     print(next_page)
#     if next_page:
#         soup = scrape_page(next_page)
#         page_data = scrape_one_page(next_page, soup)
#         df = df.append(page_data)
#         # pass
#     else:
#         break
# df.to_csv('simply_hired.csv')
# df.head()

In [25]:
job = 'Data Engineer'
location = 'texas'

url = f'https://www.simplyhired.com/search?q={job}&l={location}'
next_page = url

soup = scrape_page(url)
number_of_jobs = soup.find('div', {'data-testid':'headerSerpJobCount'}).p.text
number_per_page = len(soup.find_all('ul', {'id':'job-list'})[0]\
        .find_all('li', {'class':'css-0'}))
number_of_pages = math.ceil(int(number_of_jobs)/int(number_per_page))
print({'number_of_jobs': number_of_jobs, 'number_per_page': number_per_page, 'number_of_pages': number_of_pages})

i=1
df = pd.DataFrame(columns=['job_title', 'company', 'location', 'job_type', 'salary', 'posted_on', 'job_qualification', 'job_description'])
while next_page != None:
    print('Page Number:', i, ' Page Link: ', next_page)
    soup = scrape_page(next_page)
    page_data = scrape_one_page(next_page, soup)
    df = df.append(page_data)

    i = i+1
    next_page = get_cursor(soup, i+1)
    
print("Complete")
df.to_csv('simply_hired.csv')
df.head()

{'number_of_jobs': '407', 'number_per_page': 11, 'number_of_pages': 37}
Page Number: 1  Page Link:  https://www.simplyhired.com/search?q=Data Engineer&l=texas
https://www.simplyhired.com/search?q=Data Engineer&l=texas&job=faS8-6qI4aDvqDdybxZ9K-Kl_O2ChurBOBznSxFvQnMR9j4XTj0PFg
https://www.simplyhired.com/search?q=Data Engineer&l=texas&job=-zxIoAuyxHw5mNLZLcSqVPnuYlY-s3VxO_YQrlf6e4Xlvb5x_VqZ5Q
https://www.simplyhired.com/search?q=Data Engineer&l=texas&job=0WKAthUAS2wY_89sWDfod5gmmf0hCgDl0jvOqwlrOyMwgYTcDH5d1g
https://www.simplyhired.com/search?q=Data Engineer&l=texas&job=ESn2UspJooPZzkzCRr7oTR_z0QaDFt_85ZHJ1oK4CX33SV7BGlm6fA
https://www.simplyhired.com/search?q=Data Engineer&l=texas&job=_hV78mDPlQeGJlgol3-SVSTwLpPWztGP0H3Ejl2vXTnAX2dr5qBO8w
https://www.simplyhired.com/search?q=Data Engineer&l=texas&job=RZ3OqnMdz5S2VliOwKX0TuRJM_hLFiekrUbUq5bD3I_537KKau5hMg
https://www.simplyhired.com/search?q=Data Engineer&l=texas&job=3gxGGL4w5GngCsM3d4H2YfQ4FFQO89FbNU8TIIWBi-0VSfR65f3tNw
https://www.sim

/tmp/ipykernel_33129/477855742.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(page_data)


Page Number: 2  Page Link:  https://www.simplyhired.com/search?q=Data+Engineer&l=texas&cursor=AAsAAgAWAAAAAAAAAAAAAAACCRy%2B%2BQEBAQYB%2BVk%2BcCh8PUoOAgZ7YNGDzoq9GyDvrCamrmHw%2BL1wyWj%2B4jAocwuTeW4KcVWjXeGy
https://www.simplyhired.com/search?q=Data+Engineer&l=texas&cursor=AAsAAgAWAAAAAAAAAAAAAAACCRy%2B%2BQEBAQYB%2BVk%2BcCh8PUoOAgZ7YNGDzoq9GyDvrCamrmHw%2BL1wyWj%2B4jAocwuTeW4KcVWjXeGy&job=VAyRlfxvWjCXl3AhD_bWYB4h5fHD457y-Pt73GBjzINYI5kSmOEoFA
https://www.simplyhired.com/search?q=Data+Engineer&l=texas&cursor=AAsAAgAWAAAAAAAAAAAAAAACCRy%2B%2BQEBAQYB%2BVk%2BcCh8PUoOAgZ7YNGDzoq9GyDvrCamrmHw%2BL1wyWj%2B4jAocwuTeW4KcVWjXeGy&job=oWaclR_t83qMKOwpJy79Pr7FszvehE3kCsGQMn-oOcJ2Qxr5_IQdWw
https://www.simplyhired.com/search?q=Data+Engineer&l=texas&cursor=AAsAAgAWAAAAAAAAAAAAAAACCRy%2B%2BQEBAQYB%2BVk%2BcCh8PUoOAgZ7YNGDzoq9GyDvrCamrmHw%2BL1wyWj%2B4jAocwuTeW4KcVWjXeGy&job=hp4_yU3iJiKK3b9Bxx-nNtUQLyb49CGfZ4wmcU37qv7be9CNlH1lAg
https://www.simplyhired.com/search?q=Data+Engineer&l=texas&cursor=AAsAAgAWAAAAA

/tmp/ipykernel_33129/477855742.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(page_data)


Page Number: 3  Page Link:  https://www.simplyhired.com/search?q=Data+Engineer&l=texas&cursor=AAsAAwAhAAAAAAAAAAAAAAACCRy%2B%2BQECAQYQBjD%2BRuQUwxqZKipDXBwsD4WBzg1sNbI6AKt8rGwiTwk1mLCxfR1%2Fr2n8icQpY8bf6sQMQA5vX52wWJVv1g%3D%3D
https://www.simplyhired.com/search?q=Data+Engineer&l=texas&cursor=AAsAAwAhAAAAAAAAAAAAAAACCRy%2B%2BQECAQYQBjD%2BRuQUwxqZKipDXBwsD4WBzg1sNbI6AKt8rGwiTwk1mLCxfR1%2Fr2n8icQpY8bf6sQMQA5vX52wWJVv1g%3D%3D&job=z7_o5-eg9pDXiwe_3RvwXwDJn2LbpIZhkWmEgYW6PlKNuNDrU4M2Sg
https://www.simplyhired.com/search?q=Data+Engineer&l=texas&cursor=AAsAAwAhAAAAAAAAAAAAAAACCRy%2B%2BQECAQYQBjD%2BRuQUwxqZKipDXBwsD4WBzg1sNbI6AKt8rGwiTwk1mLCxfR1%2Fr2n8icQpY8bf6sQMQA5vX52wWJVv1g%3D%3D&job=ThYj28cCvjeaNFzLrRz-kCYzKA3eW110q-c4D8XJIwxIDQqX61IXvg
https://www.simplyhired.com/search?q=Data+Engineer&l=texas&cursor=AAsAAwAhAAAAAAAAAAAAAAACCRy%2B%2BQECAQYQBjD%2BRuQUwxqZKipDXBwsD4WBzg1sNbI6AKt8rGwiTwk1mLCxfR1%2Fr2n8icQpY8bf6sQMQA5vX52wWJVv1g%3D%3D&job=6YohuW2Ts5AeIX10dcdPXsKL8i97F6MZC5whz9PjDyLjBGVt3Yenmg

...

Page Number: 35  Page Link:  https://www.simplyhired.com/search?q=Data+Engineer&l=texas&cursor=AAsAIwGBAAAAAAAAAAAAAAACCRy%2B%2BQEBAQgBDmvLmifMwH%2Fkx4JjOFUvCaagRvH5%2FUZ5juRU7UkhunBE8QNEhYk%2F8dSziSe8
https://www.simplyhired.com/search?q=Data+Engineer&l=texas&cursor=AAsAIwGBAAAAAAAAAAAAAAACCRy%2B%2BQEBAQgBDmvLmifMwH%2Fkx4JjOFUvCaagRvH5%2FUZ5juRU7UkhunBE8QNEhYk%2F8dSziSe8&job=X-EbXUv88w1q7DfWPnRPzGcHehq3FVrEIy1cJRbsnWPf8Gl4TDdv9Q
https://www.simplyhired.com/search?q=Data+Engineer&l=texas&cursor=AAsAIwGBAAAAAAAAAAAAAAACCRy%2B%2BQEBAQgBDmvLmifMwH%2Fkx4JjOFUvCaagRvH5%2FUZ5juRU7UkhunBE8QNEhYk%2F8dSziSe8&job=qViOkui3lzjLAVUEVtFIx4sa4A6nLnOiY2BvsvyqMZevEvZ8nkfHyw
https://www.simplyhired.com/search?q=Data+Engineer&l=texas&cursor=AAsAIwGBAAAAAAAAAAAAAAACCRy%2B%2BQEBAQgBDmvLmifMwH%2Fkx4JjOFUvCaagRvH5%2FUZ5juRU7UkhunBE8QNEhYk%2F8dSziSe8&job=xhlcnGKCjqzsIkXlOEN2QkeSFDLInGr2UwTDUOv_jnC8U7oLDJXh8A
https://www.simplyhired.com/search?q=Data+Engineer&l=texas&cursor=AAsAIwGBAAAAAAAAAAAAAAACCRy%2B%2BQEB

/tmp/ipykernel_33129/477855742.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(page_data)


Page Number: 36  Page Link:  https://www.simplyhired.com/search?q=Data+Engineer&l=texas&cursor=AAsAJAGMAAAAAAAAAAAAAAACCRy%2B%2BQEAFSc0vM1MW9HHsBbvuj0vIpuOOjFAK2HAsV%2BNDcjG
https://www.simplyhired.com/search?q=Data+Engineer&l=texas&cursor=AAsAJAGMAAAAAAAAAAAAAAACCRy%2B%2BQEAFSc0vM1MW9HHsBbvuj0vIpuOOjFAK2HAsV%2BNDcjG&job=re7TBxmFOMAGTqOSSRqL2SuVZeYBOQj_5KwvUtYaGyi39oTRCvUivQ
https://www.simplyhired.com/search?q=Data+Engineer&l=texas&cursor=AAsAJAGMAAAAAAAAAAAAAAACCRy%2B%2BQEAFSc0vM1MW9HHsBbvuj0vIpuOOjFAK2HAsV%2BNDcjG&job=CnMInyHBu_w8Uhk5wO8RslLq6xe2CPRSMnm7bEx7pMBDMR2qOTXO2Q
https://www.simplyhired.com/search?q=Data+Engineer&l=texas&cursor=AAsAJAGMAAAAAAAAAAAAAAACCRy%2B%2BQEAFSc0vM1MW9HHsBbvuj0vIpuOOjFAK2HAsV%2BNDcjG&job=bU3pWmr9TruAabGgpH3jmwQgfcyLaKGyaCoaGZJszWOJ2-HP6qLdgw
https://www.simplyhired.com/search?q=Data+Engineer&l=texas&cursor=AAsAJAGMAAAAAAAAAAAAAAACCRy%2B%2BQEAFSc0vM1MW9HHsBbvuj0vIpuOOjFAK2HAsV%2BNDcjG&job=fP26dDuu2fnaiFwFQZxx5x5wzE5xaU-z1nYjYs1J7Bzj1lh9Z0S8-Q
https://ww

/tmp/ipykernel_33129/477855742.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(page_data)


,job_title,company,location,job_type,salary,posted_on,job_qualification,job_description
0,Junior Data Engineer,HYR Global Source Inc,"Dallas, TX",Contract,Estimated: $77.3K - $97.9K a year,6 days ago,Microsoft Windows Server\nWindows\nAWS\nScript...,Job Title - Junior Data Engineer\n\nLocation -...
1,Lead Big Data Engineer,ITgen systems,"Dallas, TX",Contract,$60 - $70 an hour,12 hours ago,Azure\nSpark\nApache Hive\nSoftware developmen...,Job Title: Lead Big Data Engineer\nLocation: D...
2,Sports Data Engineer,Btechnical Group,"Frisco, TX",Full-time | Contract,"$150,000 - $175,000 a year",9 days ago,Computer Science\n5 years\nMicrosoft SQL Serve...,"We, a leader in the Sports and Entertainment s..."
3,Data Engineer,Garner Health - 1.0,"Dallas, TX",,"$100,000 - $145,000 a year",,Big data\nSQL\nAWS\nPostgreSQL\nTerraform\nRed...,Garner's mission is to transform the healthcar...
4,Associate Data Engineer,48forty Solutions - 2.8,"Houston, TX",Full-time,Estimated: $66.2K - $83.8K a year,10 hours ago,CI/CD\nPower BI\nAzure\nDevOps\nSpark\nTest au...,48forty Solutions is the largest pallet manage...


In [ ]:
job_qualification

In [ ]:
# last_url = 'https://www.simplyhired.com/search?q=Data+Engineer&l=texas&cursor=AAsAJQGXAAAAAAAAAAAAAAACCP8v8AEAASKREim0qth6U1UGx3Kdm2xpc3Q%3D'
url = 'https://www.simplyhired.com/search?q=Data+Engineer&l=texas&cursor=AAsAAQALAAAAAAAAAAAAAAACCP8v8AEACLxbmdcgaMSCrKocXGxPFPTCzR2ug7Ssp0Xo'
soup = scrape_page(url)
soup.find('a', {'class':'chakra-link css-1wxsdwr', 'aria-label': f'page {3}'})
# job_title = job.h2.text.strip()
# company = job.find('span', {'data-testid': 'viewJobCompanyName'}).text if job.find('span', {'data-testid': 'viewJobCompanyName'}) else ""
# location = job.find('span', {'data-testid': 'viewJobCompanyLocation'}).text if job.find('span', {'data-testid': 'viewJobCompanyLocation'}) else ""
# job_type = job.find('span', {'data-testid': 'viewJobBodyJobDetailsJobType'}).text if job.find('span', {'data-testid': 'viewJobBodyJobDetailsJobType'}) else ""
# salary = job.find('span', {'data-testid': 'viewJobBodyJobCompensation'}).text if job.find('span', {'data-testid': 'viewJobBodyJobCompensation'}) else ""
# posted_on = job.find('span', {'data-testid': 'viewJobBodyJobPostingTimestamp'}).text if job.find('span', {'data-testid': 'viewJobBodyJobPostingTimestamp'}) else ""
# job_qualification = list(job.find('div', {'data-testid': 'viewJobQualificationsContainer'})\
#                                 .find('ul').strings)
# job_qualification = "\n".join(job_qualification)

# job_description = list(job.find('div', {'data-testid':'viewJobBodyJobFullDescriptionContent'}).strings)
# job_description = "\n".join(job_description)
# data = {
#         'job_title':job_title,
#         'company':company,
#         'location': location, 
#         'job_type': job_type, 
#         'salary':salary, 
#         'posted_on':posted_on, 
#         'job_qualification':job_qualification, 
#         'job_description': job_description
#         }

# data = {
#         'job_title':job_title,
#         'company':company,
#         'location': location, 
#         'job_type': job_type, 
#         'salary':salary, 
#         'posted_on':posted_on, 
#         'job_qualification':job_qualification, 
#         'job_description': job_description
#         }
# print(data)

In [ ]:
# def scrape_one_page(soup):
#     ul = soup.find_all('ul', id = 'job-list')
#     li = ul[0].find_all('li', {'class':'css-0'})
#     job_links = get_job_links(li)
#     data_list = []
#     for job_link in job_links:
#         print(job_link)
#         job = scrape_page(job_link)
#         job_details = job.find_all('span', {'data-testid':'detailText'})
#         job_title = job.h2.text.strip()
#         company = job_details[0].text
#         location = job_details[1].text
#         job_type = job_details[2].text
#         salary = job_details[3].text
#         posted_on = job_details[4].text if len(job_details)>4 else ''

#         job_qualification = list(job.find('div', {'data-testid': 'viewJobQualificationsContainer'})\
#                                 .find('ul').strings)
#         job_qualification = "\n".join(job_qualification)

#         job_description = list(job.find('div', {'data-testid':'viewJobBodyJobFullDescriptionContent'}).strings)
#         job_description = "\n".join(job_description)
#         data = {
#                 'job_title':job_title,
#                 'company':company,
#                 'location': location, 
#                 'job_type': job_type, 
#                 'salary':salary, 
#                 'posted_on':posted_on, 
#                 'job_qualification':job_qualification, 
#                 'job_description': job_description
#                 }
#         data_list.append(data)
#         # break
#     return data_list

In [ ]:
# job_header = job.find('div', {'data-testid':'viewJobHeadingContainer'})
# job_title = job_header.h2.text
# company_location = job_header.find_all('span', {'data-testid': 'detailText'})
# company = company_location[0].text
# location = company_location[1].text
# print(job_title, company, location)
